\* 이 글에서는 `Scheme`을 이용하여 설명하겠습니다.

__내가 그의 이름을 불러주었을 때__


__그는 나에게로 와서__


__꽃이 되었다.__

( 김춘수의 시 '꽃' 중에서 )

위의 시에서 '그의 이름'은 '어떤 타자'를 지칭하는 의미입니다.

그런데 만약, '-그의 이름-이라고 말해봐'하고 말하면 거기서 -그의 이름-은 __글자 그대로__ -그의 이름-이 됩니다.

이처럼 '글자 그대로'를 데이터로 다룰 때 그것을 __글자 데이터( Symbolic data )__라고 합니다.

여기서는 글자 데이터를 가지고 만든 식을 미분하는 코드를 작성하겠습니다.

먼저, 미분을 할 때 사용하는 규칙을 정리하면,

### 1. $\frac{dc}{dx} = 0$

### 2. $\frac{dx}{dx} = 1$

### 3. $\frac{d(u+v)}{dx} = \frac{d(u)}{dx} + \frac{d(v)}{dx}$

### 4. $\frac{d(uv)}{dx} = u(\frac{dv}{dx}) + v(\frac{du}{dx})$

아래의 모듈은 들어온 데이터가 

1) 숫자인지( 숫자가 아니면 글자로 인식 ) 판단

2) 1.과 2.에 해당하는지

3) 그게 아니면 3.과 4.의 정의대로 글자식으로 만듭니다.

( \* 이 코드는 아래에서 더 보완합니다. )

3.과 4.에 대한 코드는 모듈로 아래와 같습니다.

In [1]:
;; Judge exp is a number?(not symbol?) and num = exp?
(define (=number? exp num)
  (and (number? exp) (= exp num)))


;; 3. make-sum
(define (make-sum a1 a2)
  (cond ((=number? a1 0) a2)
        ((=number? a2 0) a1)
        ((and (number? a1) (number? a2)) (+ a1 a2))
        (else (list '+ a1 a2))))


;; 4. make-product
(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
        ((=number? m1 1) m2)
        ((=number? m2 1) m1)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list '* m1 m2))))

그다음, 대수식에서 사용할 기본 프로시저를 나열하겠습니다.


간략하게 설명하면,
\* `augend`는 덧셈수에서 첫째 마디, `addend`는 덧셈수에서 둘째 마디입니다. 

(예를들어, `a1 + a2`이면 `a1`은 `augend`, `a2`는 `addend`입니다.)  

\* 마찬가지로 `multiplicand`는 곱셈수에서 첫째 마디, `multiplier입니다`는 곱셈수에서 둘째 마디입니다. 

(예를들어, `m1 * m2`이면 `m1`은 `multiplicand`, `m2`는 `multiplier`입니다.)

In [2]:
(define (variable? x) (symbol? x))
(define (same-variable? v1 v2)
  (and (variable? v1)
       (variable? v2)
       (eq? v1 v2)))
(define (sum? x)
  (and (pair? x) (eq? (car x) '+)))
(define (addend s) (cadr s))
(define (augend s) (caddr s))
(define (product? x)
  (and (pair? x) (eq? (car x) '*)))
(define (multiplier p) (cadr p))
(define (multiplicand p) (caddr p))

이제, 여기서 구현해볼 미분 규칙을 정리하면,

### 5. $\frac{d(u^n)}{dx} = nu^{n-1}(\frac{du}{dx})$

5.를 구현하기 위해서는, 위에서 만든 `make-sum`, `make-product`처럼 지수를 계산하는 `make-exponentiation`이 필요합니다.


여기에 지수인지 확인하는 `exponentiation?`, 식에서 밑과 지수를 분리하는 `base`, `exponent`도 만들면,

In [3]:
(define (base x)
  (cadr x))


(define (exponent x)
  (caddr x))


(define (exponentiation? x)
  (and (pair? x) (eq? (car x) '**)))  


(define (make-exponentiation base exponent)
  (cond ((=number? exponent 0) 1)
        ((=number? exponent 1) base)
        (else (list '** base exponent))))

5.의 규칙을 살펴보면,

$\frac{d(u^n)}{dx} = nu^{n-1}(\frac{du}{dx})$

세 수의 곱으로 되어있음을 알 수 있습니다. 따라서 이것은 `make-product`로 표현할 수 있습니다.

__`make-product`$(n, u^{n-1}, \frac{du}{dx}$)__

이런 방법으로 계속 연산자를 함수화하면,

__`make-product`$(n,$ `make-exponentiation`$(u^{n-1})$, `derivation`$(\frac{du}{dx})$)__

다만, 코드의 구현상 make-product는 2개의 마디(항)를 가지므로, 

__`make-product`(`make-product`$(n,$ `make-exponentiation`$(u^{n-1})$), `derivation`$(\frac{du}{dx})$))__

이에 대한 핵심 부분을 구현하면,

In [ ]:
...
(cond ...
    ((exponentiation? exp)
     (if (number? (exponent exp))
         (make-product (make-product (exponent exp)
                                     (make-exponentiation (base exp)
                                                          (- (exponent exp) 1))
                                     )
                       (deriv (base exp) var))
         (make-product (make-product (exponent exp)
                                     (make-exponentiation (base exp)
                                                          (list '- (exponent exp) 1))
                                     )
                       (deriv (base exp) var))))
                       ...

`deriv`의 전체 구현은 아래와 같습니다.

https://github.com/my-snippet/SICP/blob/master/backup/2-Building-Abstractions-with-Data/2-3-Symbolic-Data/practice/pr-2-56-deriv-with-exponent.ss


In [4]:
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp)
         (if (same-variable? exp var) 1 0))
        ((sum? exp)
         (make-sum (deriv (addend exp) var)
                   (deriv (augend exp) var)))
        ((product? exp)
         (make-sum
          (make-product (multiplier exp)
                        (deriv (multiplicand exp) var))
          (make-product (deriv (multiplier exp) var)
                        (multiplicand exp))))
        ((exponentiation? exp)
         (if (number? (exponent exp))
             (make-product (make-product (exponent exp)
                                         (make-exponentiation (base exp)
                                                              (- (exponent exp) 1))
                                         )
                           (deriv (base exp) var))
             (make-product (make-product (exponent exp)
                                         (make-exponentiation (base exp)
                                                              (list '- (exponent exp) 1))
                                         )
                           (deriv (base exp) var))))
        (else
         (error "unknown expression type -- DERIV" exp))))

여기서 '지수가 숫자인가, 문자인가'에 따라 어디까지를 계산하고 어디까지를 글자식으로 표현할지 결정합니다.

( \* 그리고 이 부분이 구현을 다소 중복되게 만들었습니다. 이 부분에 대한 함수화를 거치면 코드를 더 간결하게 만들 수 있을 것입니다. )

아래의 몇가지 경우에 대해 검증해보면,

### $\frac{d(x^0)}{dx}$, $\frac{d(x^1)}{dx}$, $\frac{d(x^2)}{dx}$, $\frac{d(x^3)}{dx}$, $\frac{d(x^4)}{dx}, \frac{d(x^n)}{dx}$

In [5]:
(deriv '(** x 0) 'x)

0

In [6]:
(deriv '(** x 1) 'x)

1

In [7]:
(deriv '(** x 2) 'x)

(* 2 x)

In [8]:
(deriv '(** x 3) 'x)

(* 3 (** x 2))

In [9]:
(deriv '(** x 4) 'x)

(* 4 (** x 3))

In [10]:
(deriv '(** x n) 'x)

(* n (** x (- n 1)))

### $\frac{d(y^2)}{dx}$, $\frac{d(xy)}{dx}$, $\frac{d(uy)}{dx}$

In [11]:
(deriv '(** y 2) 'x)

0

In [12]:
(deriv '(* x y) 'x)

y

In [13]:
(deriv '(* u y) 'x)

0

### $\frac{d(y^n)}{dx}$, $\frac{d(uy)^n}{dx}$

In [14]:
(deriv '(** y n) 'x)

0

In [15]:
(deriv '(** (* u y) n) 'x)

0

하지만 이 프로그램은 다양한 표현이 가능한 __'문자를 다루는 것'__이기 때문에, 여러 문제점을 가지고 있습니다.

예를 들면, 아래와 같은 경우입니다.

### a) $\frac{d(3x)^n}{dx}$, b) $\frac{d(xy)^n}{dx}$

이 경우 문제가 되는 부분은

a) 1이 아닌 상수가 들어왔을 때의 예외처리가 없다는 것과,

b) 문자상수와 변수의 곱 형태를 제곱했을 때, 문자상수의 제곱은 건드리지 말아야(`make-exponentiation` 연산 중) 하는게 그것을 분리하는 예외처리가 없다는 것입니다.

결과를 확인해보면,

In [16]:
(deriv '(** (* 3 x) n) 'x)

(* (* n (** (* 3 x) (- n 1))) 3)

In [17]:
(deriv '(** (* x y) n) 'x)

(* (* n (** (* x y) (- n 1))) y)

문자열을 파악하는 프로그램을 작성하는 것은 (다양한 표현이 가능해서) 간소화(미분 연산을 위해 식을 합, 곱, 제곱의 세가지로 쪼개서 계속 돌리듯) 구현과 다양한 예외처리가 요구된다는 것을 간단한 미분 예제를 통해 알 수 있었습니다.

__\* 글을 쓰는 도중에도 구현 범위를 넘어서는 예외 상황이 많이 떠올랐습니다. 여러 사람이 코드를 검사하고, 또 기계가 코드 구석구석을 통합적으로 검사할 수 있다면 이런 오류는 많이 줄일 수 있을 것 같다고 생각을 했습니다.__

다음 기회에 예외처리 구현 및 아래의 다변수미분 형태를 구현하도록 하겠습니다.

### 6. $\frac{d(u+v+w+...)}{dx} = \frac{d(u)}{dx} + \frac{d(v)}{dx} + \frac{d(w)}{dx} +\ ...$

### 7. $\frac{d(uvw...)}{dx} = uw...(\frac{dv}{dx}) + vw...(\frac{du}{dx}) + uv...(\frac{dw}{dx})$